In [1]:
import numpy as np
import pandas as pd
import openpyxl

In [2]:
df_asos = pd.read_excel("../DABA/META_관측지점정보_ASOS.xlsx", header=1)
print("before: ", df_asos.shape)
df_asos = df_asos.drop_duplicates(['지점명'], keep='first')
print("after: ", df_asos.shape)

df_asos_ext = df_asos[['지점','지점명','위도','경도']]
df_asos_ext['관측소'] = 'ASOS'
df_asos_ext.head()

before:  (141, 13)
after:  (107, 13)


C:\Users\tjoeun\AppData\Local\Temp\ipykernel_14524\4293557183.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_asos_ext['관측소'] = 'ASOS'


,지점,지점명,위도,경도,관측소
0,90,속초,38.2509,128.5647,ASOS
1,93,북춘천,37.9474,127.7544,ASOS
2,95,철원,38.1479,127.3042,ASOS
3,98,동두천,37.9019,127.0607,ASOS
4,99,파주,37.8859,126.7665,ASOS


In [3]:

df_aws = pd.read_excel("../DABA/META_관측지점정보_AWS.xlsx", header=1)
print("before: ", df_aws.shape)
df_aws = df_aws.drop_duplicates(['지점명'], keep='first')
print("after: ", df_aws.shape)

df_aws_ext = df_aws[['지점','지점명','위도','경도']]
df_aws_ext['관측소'] = 'AWS'
df_aws_ext.head()


before:  (2173, 13)
after:  (713, 13)


C:\Users\tjoeun\AppData\Local\Temp\ipykernel_14524\1998806762.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aws_ext['관측소'] = 'AWS'


,지점,지점명,위도,경도,관측소
0,12,안면도(감),36.5333,126.3167,AWS
1,96,독도,37.2395,131.8698,AWS
3,116,관악(레),37.4453,126.9640,AWS
5,144,군산(레),36.0123,126.7834,AWS
6,160,부산(레),35.1188,129.0000,AWS


In [4]:
df_mtn_aws = pd.read_csv("../DABA/산악AWS_지점상세정보.csv", encoding='cp949')
print("before: ", df_mtn_aws.shape)
df_mtn_aws = df_mtn_aws.drop_duplicates(['위도','경도'], keep='first')
print("after: ", df_mtn_aws.shape)

df_mtn_aws_ext = df_mtn_aws[['지점번호','산이름','위도','경도']]
df_mtn_aws_ext.columns = ['지점','지점명','위도','경도']
df_mtn_aws_ext['관측소'] = 'MTN'
df_mtn_aws_ext.head()

before:  (387, 6)
after:  (387, 6)


C:\Users\tjoeun\AppData\Local\Temp\ipykernel_14524\782829170.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mtn_aws_ext['관측소'] = 'MTN'


,지점,지점명,위도,경도,관측소
0,1890,파주 팔일봉,37.78,126.92,MTN
1,1891,양평 금왕산,37.46,127.79,MTN
2,1892,양평 어비산,37.57,127.53,MTN
3,1893,양평 삼각산,37.44,127.71,MTN
4,1894,포천 한나무봉,37.95,127.38,MTN


In [5]:
all_stn_df = pd.concat([df_asos_ext, df_aws_ext, df_mtn_aws_ext])
print(all_stn_df.shape)
all_stn_df = all_stn_df.set_index('지점명')
all_stn_df.head()


(1207, 5)


,지점,위도,경도,관측소
지점명,,,,
속초,90,38.2509,128.5647,ASOS
북춘천,93,37.9474,127.7544,ASOS
철원,95,38.1479,127.3042,ASOS
동두천,98,37.9019,127.0607,ASOS
파주,99,37.8859,126.7665,ASOS


In [6]:
import pandas as pd
import folium


# 위경도 영역 설정
p1_lat, p1_lon = 38.05, 128.4
p2_lat, p2_lon = 38.15, 128.4
p3_lat, p3_lon = 38.15, 128.5
p4_lat, p4_lon = 38.05, 128.5
p5_lat, p5_lon = 38.05, 128.4


# 위도, 경도
lat, lon = 38., 128.4
# 줌 크기
zoom_size = 12

# 구글 지도 타일 설정
tiles = "http://mt0.google.com/vt/lyrs=p&hl=ko&x={x}&y={y}&z={z}"
# 속성 설정
attr = "Google"
# 지도 객체 생성
m = folium.Map(location = [lat, lon],
               zoom_start = zoom_size,
               tiles = tiles,
               attr = attr)

for name, lat, lon, stnif in zip(all_stn_df.index, all_stn_df.위도, all_stn_df.경도, all_stn_df.관측소):
   if stnif == 'ASOS':
       color = 'red'
   elif stnif == 'AWS':
       color = 'red'
   elif stnif == 'MTN':    
       color = 'green'
   else:
       color = 'white'  # 미확인
   folium.Marker( [lat, lon], popup=name, icon=folium.Icon(color = color) ).add_to(m) # popup: 팝업창, 마우스 클릭 정보표시

folium.PolyLine([
    [p1_lat, p1_lon],
    [p2_lat, p2_lon],
    [p3_lat, p3_lon],
    [p4_lat, p4_lon],
    [p5_lat, p5_lon]
], color='red').add_to(m)
m